# Master Discogs Database

In [46]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import masterdb


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-05-29 20:39:30.291942


In [47]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Counter({'Discogs': 3919, 'AllMusic': 3887, 'MusicBrainz': 3674, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 175, 'CDandLP': 80, 'RateYourMusic': 30, 'MusicStack': 5, 'MetalStorm': 0})


In [48]:
dbdata = {}
keys   = ["Artists", "Artist", "Utils"]

### Discogs
from artistsDC import artistsDC
from artistDC import artistDC
from discogsUtils import discogsUtils
dbinfo = [artistsDC, artistDC, discogsUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["Discogs"] = dbinfo

### AllMusic
from artistsAM import artistsAM
from artistAM import artistAM
from discogsUtils import allmusicUtils
dbinfo = [artistsAM, artistAM, allmusicUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["AllMusic"] = dbinfo

### MusicBrainz
from artistsMB import artistsMB
from artistMB import artistMB
from discogsUtils import musicbrainzUtils
dbinfo = [artistsMB, artistMB, musicbrainzUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["MusicBrainz"] = dbinfo

## AceBootlegs
from artistAB import artistAB
from artistsAB import artistsAB
from discogsUtils import acebootlegsUtils
dbinfo = [artistsAB, artistAB, acebootlegsUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["AceBootlegs"] = dbinfo

## RateYourMusic
from artistRM import artistRM
from artistsRM import artistsRM
from discogsUtils import rateyourmusicUtils
dbinfo = [artistsRM, artistRM, rateyourmusicUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["RateYourMusic"] = dbinfo

## LastFM
from artistLM import artistLM
from artistsLM import artistsLM
from discogsUtils import lastfmUtils
dbinfo = [artistsLM, artistLM, lastfmUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["LastFM"] = dbinfo

## DatPiff
from artistDP import artistDP
from artistsDP import artistsDP
from discogsUtils import datpiffUtils
dbinfo = [artistsDP, artistDP, datpiffUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["DatPiff"] = dbinfo

## RockCorner
from artistRC import artistRC
from artistsRC import artistsRC
from discogsUtils import rockcornerUtils
dbinfo = [artistsRC, artistRC, rockcornerUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["RockCorner"] = dbinfo

## CDandLP
from artistCL import artistCL
from artistsCL import artistsCL
from discogsUtils import cdandlpUtils
dbinfo = [artistsCL, artistCL, cdandlpUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["CDandLP"] = dbinfo

## MusicStack
from artistMS import artistMS
from artistsMS import artistsMS
from discogsUtils import musicstackUtils
dbinfo = [artistsMS, artistMS, musicstackUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["MusicStack"] = dbinfo

## MetalStorm
from artistMT import artistMT
from artistsMT import artistsMT
from discogsUtils import metalstormUtils
dbinfo = [artistsMT, artistMT, metalstormUtils]
dbinfo = dict(zip(keys, dbinfo))
dbdata["MetalStorm"] = dbinfo

## General
from discogsBase import discogs
for db in dbdata.keys():
    print("Creating DB Info For {0}".format(db))
    dbdata[db]["Disc"]    = discogs(db.lower())
    dbdata[db]["Artist"]  = dbdata[db]["Artist"](dbdata[db]["Disc"])
    dbdata[db]["Artists"] = dbdata[db]["Artists"](dbdata[db]["Disc"])
    dbdata[db]["Utils"]   = dbdata[db]["Utils"]()

Creating DB Info For Discogs
Creating DB Info For AllMusic
Creating DB Info For MusicBrainz
Creating DB Info For AceBootlegs
Creating DB Info For RateYourMusic
Creating DB Info For LastFM
Creating DB Info For DatPiff
Creating DB Info For RockCorner
Creating DB Info For CDandLP
Creating DB Info For MusicStack
Creating DB Info For MetalStorm


In [55]:
%load_ext autoreload
%autoreload

dbs = dbdata.keys()
#dbs = ["DatPiff", "RockCorner", "CDandLP", "MusicStack", "MetalStorm"]
#dbs = ["AceBootlegs"]

########################################################################################################
## Loop over DBs
########################################################################################################
for db in dbs:
    print("\n")
    print("="*125)
    print("="*125)
    print("=",db)
    print("="*125)
    print("="*125)
    print("\n")
    
    
    disc  = dbdata[db]["Disc"]
    mymdb = masterdb(db, disc, force=True)
    mymdb.setMyMusicDB(mdb)
    
    
    #################################
    # Artists
    #################################
    mymdb.createArtistIDMap()
    slimArtistDB      = mymdb.getSlimArtistDB()
    print("All   --> {0}".format(slimArtistDB.shape))
    knownSlimArtistDB = mymdb.getKnownSlimArtistDB()
    print("Known --> {0}".format(knownSlimArtistDB.shape))
        
    
    #################################
    # Artist Albums
    #################################
    mymdb.createArtistAlbumIDMap()
    slimArtistAlbumsDB      = mymdb.getSlimArtistAlbumsDB()
    print("All Albums    --> {0}".format(slimArtistAlbumsDB.shape))
    knownSlimArtistAlbumsDB = mymdb.getKnownSlimArtistAlbumsDB()
    print("Known Albums  --> {0}".format(knownSlimArtistAlbumsDB.shape))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


= Discogs


Current Time is Fri May 29, 2020 21:07:18 for Creating Artist DBs
/Volumes/Piggy/Discog/artists-discogs-db/metadata/0-Metadata.p  	0 7326
/Volumes/Piggy/Discog/artists-discogs-db/metadata/1-Metadata.p  	1 14910
/Volumes/Piggy/Discog/artists-discogs-db/metadata/10-Metadata.p  	2 22291
/Volumes/Piggy/Discog/artists-discogs-db/metadata/11-Metadata.p  	3 29743
/Volumes/Piggy/Discog/artists-discogs-db/metadata/12-Metadata.p  	4 37228
/Volumes/Piggy/Discog/artists-discogs-db/metadata/13-Metadata.p  	5 44747
/Volumes/Piggy/Discog/artists-discogs-db/metadata/14-Metadata.p  	6 52016
/Volumes/Piggy/Discog/artists-discogs-db/metadata/15-Metadata.p  	7 59416
/Volumes/Piggy/Discog/artists-discogs-db/metadata/16-Metadata.p  	8 66737
/Volumes/Piggy/Discog/artists-discogs-db/metadata/17-Metadata.p  	9 74164
/Volumes/Piggy/Discog/artists-discogs-db/metadata/18-Metadata.p  	10 81609
/Volumes/Piggy/Disco

/Volumes/Piggy/Discog/artists-discogs-db/metadata/99-Metadata.p  	99 738730



Saving 738730 entries to /Volumes/Piggy/Discog/db-discogs/ArtistIDToRef.p

Saving data to /Volumes/Piggy/Discog/db-discogs/ArtistIDToRef.p
  --> This file is 16.8MB.
Saved data to /Volumes/Piggy/Discog/db-discogs/ArtistIDToRef.p
  --> This file is 16.8MB.
Saving 738730 entries to /Volumes/Piggy/Discog/db-discogs/ArtistIDToName.p

Saving data to /Volumes/Piggy/Discog/db-discogs/ArtistIDToName.p
  --> This file is 14.1MB.
Saved data to /Volumes/Piggy/Discog/db-discogs/ArtistIDToName.p
  --> This file is 14.1MB.
Saving 738730 entries to /Volumes/Piggy/Discog/db-discogs/ArtistIDToVariations.p

Saving data to /Volumes/Piggy/Discog/db-discogs/ArtistIDToVariations.p
  --> This file is 27.5MB.
Saved data to /Volumes/Piggy/Discog/db-discogs/ArtistIDToVariations.p
  --> This file is 27.5MB.
Current Time is Fri May 29, 2020 21:08:40 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 1.4 minu

/Volumes/Piggy/Discog/artists-discogs-db/metadata/6-MediaMetadata.p 	420259    925706    3924319   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/60-MediaMetadata.p 	427662    942583    3997908   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/61-MediaMetadata.p 	435015    959873    4068511   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/62-MediaMetadata.p 	442486    976498    4135348   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/63-MediaMetadata.p 	449952    992115    4205433   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/64-MediaMetadata.p 	457470    1008422   4274258   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/65-MediaMetadata.p 	464885    1025814   4347654   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/66-MediaMetadata.p 	472277    1042461   4417069   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/67-MediaMetadata.p 	479759    1058172   4484683   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/68-MediaMetadata.p 	487017    1076468   45

/Volumes/Piggy/Discog/artists-allmusic-db/metadata/14-Metadata.p  	0 362
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/21-Metadata.p  	1 748
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/0-Metadata.p  	2 1131
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/7-Metadata.p  	3 1532
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/15-Metadata.p  	4 1892
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/22-Metadata.p  	5 2253
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/1-Metadata.p  	6 2618
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/8-Metadata.p  	7 3015
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/16-Metadata.p  	8 3409
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/23-Metadata.p  	9 3815
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/2-Metadata.p  	10 4209
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/9-Metadata.p  	11 4602
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/24-Metadata.p  	12 4979
/Volumes/Piggy/Discog/artists-allmusic-db/m

  --> This file is 834.8kB.
Saved data to /Volumes/Piggy/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 834.8kB.
Current Time is Fri May 29, 2020 21:24:09 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 5 seconds.
Current Time is Fri May 29, 2020 21:24:09 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 38884 Artists
	Shape --> (38884, 1)
  Finding Real Artist Name
	Shape --> (38884, 3)
  Removing None Artist
	Shape --> (38884, 3)
  Finding Disc Artist Name
	Shape --> (38884, 4)
DataFrame Shape is (38884, 4)
Current Time is Fri May 29, 2020 21:24:10 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 817.0 millseconds.
Saving Master Artist DB File: /Volumes/Piggy/Discog/db-all

...

/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/0-Metadata.p  	0 64
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/1-Metadata.p  	1 128
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/10-Metadata.p  	2 189
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/11-Metadata.p  	3 234
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/12-Metadata.p  	4 276
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/13-Metadata.p  	5 328
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/14-Metadata.p  	6 363
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/15-Metadata.p  	7 408
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/16-Metadata.p  	8 454
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/17-Metadata.p  	9 498
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/18-Metadata.p  	10 543
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/19-Metadata.p  	11 584
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/2-Metadata.p  	12 630
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/20-Metadata.p  

DataFrame Shape is (4977, 4)
Found 4084 artist entries
Counter({'Discogs': 3919, 'AllMusic': 3887, 'MusicBrainz': 3674, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 175, 'CDandLP': 80, 'RateYourMusic': 30, 'MusicStack': 5, 'MetalStorm': 0})
DataFrame Shape is (75, 4)
Saving Master Known Artist DB File: /Volumes/Piggy/Discog/db-cdandlp/MasterKnownSlimArtistDB.p
  --> This file is 3.4kB.
Known --> (75, 4)
Current Time is Fri May 29, 2020 21:27:39 for Creating Artist DBs
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/0-MediaMetadata.p 	64        409       409       
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/1-MediaMetadata.p 	128       866       866       
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/10-MediaMetadata.p 	189       1349      1349      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/11-MediaMetadata.p 	234       1735      1735      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/12-MediaMetadata.p 	276       2054      2054      
/Vo

/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/78-MediaMetadata.p 	3792      30095     30095     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/79-MediaMetadata.p 	3842      30518     30518     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/8-MediaMetadata.p 	3893      30895     30895     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/80-MediaMetadata.p 	3943      31309     31309     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/81-MediaMetadata.p 	3991      31758     31758     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/82-MediaMetadata.p 	4046      32177     32177     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/83-MediaMetadata.p 	4097      32474     32474     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/84-MediaMetadata.p 	4146      32871     32871     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/85-MediaMetadata.p 	4199      33295     33295     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/86-MediaMetadata.p 	4252      33677     33

/Volumes/Piggy/Discog/artists-musicstack-db/metadata/95-Metadata.p  	78 180
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/96-Metadata.p  	79 183
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/97-Metadata.p  	80 187



Saving 187 entries to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToRef.p

Saving data to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToRef.p
  --> This file is 1.3kB.
Saved data to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToRef.p
  --> This file is 1.3kB.
Saving 187 entries to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToName.p

Saving data to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToName.p
  --> This file is 3.7kB.
Saved data to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToName.p
  --> This file is 3.7kB.
Saving 187 entries to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToVariations.p

Saving data to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToVariations.p
  --> This file is 4.3kB.
Saved data to /Volumes/Piggy/Discog/db-musicstack/ArtistIDToVari

  --> This file is 16.8kB.
Current Time is Fri May 29, 2020 21:27:46 for Done with 
=================================== Creating Artist Album DB ===================================
Process [Done with 
=================================== Creating Artist Album DB ===================================] took 78.0 millseconds.
All Albums    --> (187, 1)
Current Time is Fri May 29, 2020 21:27:46 for 
=================================== Creating Known Artist Album DB ===================================
Current Time is Fri May 29, 2020 21:27:46 for 
=================================== Creating Artist Album DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 187 Artists
	Shape --> (187, 1)
DataFrame Shape is (187, 1)
Saving Master Artist Albums DB File: /Volumes/Piggy/Discog/db-musicstack/MasterSlimArtistAlbumsDB.p
  --> This file is 16.8kB.
Current Time is Fri May 29, 2020 21:27:46 for Done with 
=================================== Creating Artist Album DB 

In [53]:
data = getFile('/Volumes/Piggy/Discog/db-acebootlegs/MasterKnownArtistAlbumsDB.p')
data

,Albums
393858009400,{'Bootleg': {'27966': 'Live Trash'}}
363272091111,{'Bootleg': {'19772': 'Live In France'}}
872005022711,{'Bootleg': {'20735': 'Live At The Greek Theat...
569585428012,"{'Bootleg': {'6670': 'K-Zew Fm', '4945': 'Holl..."
386616218212,{'Bootleg': {'24053': 'Nassau Coliseum Off Mas...
802869707213,"{'Bootleg': {'27291': 'Carnegie Hall', '24322'..."
890221689213,"{'Bootleg': {'11138': 'Malahide Castle', '1113..."
752885459014,"{'Bootleg': {'26826': 'Almost Starshipshape', ..."
367751746516,"{'Bootleg': {'28768': 'Morumbi Stadium 1981', ..."
806844302417,"{'Bootleg': {'26012': 'This Is Radio Clash', '..."


In [40]:
mdb = myMusicDBMap(debug=False)
musicmap = mdb.get()

Counter({'Discogs': 3919, 'AllMusic': 3887, 'MusicBrainz': 3674, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 175, 'CDandLP': 80, 'RateYourMusic': 30, 'MusicStack': 5, 'MetalStorm': 0})


In [ ]:
knownSlimArtistDB

In [ ]:

    
    

########################################################################################################################
#
# Artist DB
#
########################################################################################################################
def getArtistDB(disc, force=False):
    start, cmt = clock("\n=================================== Creating Artist DB ===================================")
    if force is False:
        print("Using previously created Artist DB")
        discdf = disc.getMasterArtistDiscogsDB()
        elapsed(start, cmt)
        return discdf
    
    print("Loading ArtistID Data")
    artistIDtoName  = Series(disc.getArtistIDToNameData())
    artistIDtoRef   = Series(disc.getArtistIDToRefData())
    artistIDToVariations  = Series(disc.getArtistIDToVariationsData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoName.shape[0]))
    cols = ["Name"]
    discdf = DataFrame(artistIDtoName)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Ref")
    discdf = discdf.join(DataFrame(artistIDtoRef))
    cols += ["Ref"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Variations")
    discdf = discdf.join(DataFrame(artistIDToVariations))
    cols += ["Variations"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    discdf["Known"] = True
    
    print("  Finding Real Artist Name")
    discdf[["Artist", "Num"]] = DataFrame(discdf['Name'].apply(self.realName).tolist(), index=discdf.index)
    print("\tShape --> {0}".format(discdf.shape))

    

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)

    print("Saving Master Artist DB File")
    saveFilename = disc.getMasterArtistDiscogsDBFilename()
    saveFile(ifile=saveFilename, idata=discdf, debug=False)
    
    return discdf    
    
    
    

########################################################################################################################
#
# Artist Metadata DB
#
########################################################################################################################
def getArtistMetadataDB(disc, force=True):
    start, cmt = clock("\n=================================== Creating Artist Metadata DB ===================================")
    if force is False:
        print("Using previously created Artist Metadata DB")
        discdf = disc.getMasterArtistMetadataDiscogsDB()
        elapsed(start, cmt)
        return discdf
    
    print("Loading ArtistID Data")
    artistIDtoGenre          = Series(disc.getArtistIDToGenreData())
    artistIDtoStyle          = Series(disc.getArtistIDToStyleData())
    artistIDToCollaboration  = Series(disc.getArtistIDToCollaborationData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoGenre.shape[0]))
    cols = ["Genre"]
    discdf = DataFrame(artistIDtoGenre)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Style")
    discdf = discdf.join(DataFrame(artistIDtoStyle))
    cols += ["Style"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Collaboration")
    discdf = discdf.join(DataFrame(artistIDToCollaboration))
    cols += ["Collaboration"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)

    print("Saving Master Artist Metadata DB File")
    saveFilename = disc.getMasterArtistMetadataDiscogsDBFilename()
    saveFile(ifile=saveFilename, idata=discdf, debug=False)
    
    return discdf




########################################################################################################################
#
# Artist Albums DB
#
########################################################################################################################
def getArtistAlbumsDB(disc, loadRefs=False, force=False):
    start, cmt = clock("\n=================================== Creating Artist Albums DB ===================================")
    if force is False:
        print("Using previously created Artist Albums DB")
        discdf = disc.getMasterArtistAlbumsDiscogsDB()
        elapsed(start, cmt)
        return discdf
    
    print("Loading ArtistID Data")
    artistIDtoAlbumNames  = Series(disc.getArtistIDToAlbumNamesData())
    if loadRefs:
        artistIDtoAlbumRefs   = Series(disc.getArtistIDToAlbumRefsData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoAlbumNames.shape[0]))
    cols = ["Albums"]
    discdf = DataFrame(artistIDtoAlbumNames)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    
    print("Saving Master Artist Albums DB File")
    saveFilename = disc.getMasterArtistAlbumsDiscogsDBFilename()
    saveFile(ifile=saveFilename, idata=discdf, debug=False)
    
    elapsed(start, cmt)
        
    return discdf






########################################################################################################################
#
# Artist Album Known DB
#
########################################################################################################################
def getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB):
    start, cmt = clock("\n=================================== Creating Artist Album DB ===================================")
    from pandas import Series, DataFrame
    
    idx=discAlbumDB.index
    
    tmpdb = discArtistAlbumsDB["Albums"].copy()
    print("Creating Pandas DataFrame for {0} Arist Albums".format(tmpdb.shape[0]))
    discdf = DataFrame(tmpdb.apply(isKnownAlbum, idx=idx).tolist(), index=tmpdb.index)
    discdf.columns = ["Known Albums", "All Albums", "Albums"]
    print("\tShape --> {0}".format(discdf.shape))
    
    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)
    
    return discdf
    
def isKnownAlbum(x, **kwargs):
    retval = {}
    albumSummary = [0, 0]
    for mediaType in x.keys():
        for albumID in x[mediaType].keys():
            albumName = x[mediaType][albumID]
            #print(mediaType,albumID,albumName,'\t\t',end="")
            known     = albumID in kwargs['idx']
            #print(known)
            
            retval[albumID] = [albumName, mediaType, known]
            albumSummary[0] += known
            albumSummary[1] += 1
            
    return [albumSummary[0], albumSummary[1], retval]





########################################################################################################################
#
# Album DB
#
########################################################################################################################
def getAlbumDB(disc):
    start, cmt = clock("\n=================================== Creating Artist Album DB ===================================")
    from pandas import Series, DataFrame
    print("Loading AlbumID Data")
    albumIDtoName    = Series(disc.getAlbumIDToNameData())
    albumIDtoRef     = Series(disc.getAlbumIDToRefData())
    albumIDToArtists = Series(disc.getAlbumIDToArtistsData())

    print("Creating Pandas DataFrame for {0} Albums".format(albumIDtoName.shape[0]))
    cols = ["Name"]
    discdf = DataFrame(albumIDtoName)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Ref")
    discdf = discdf.join(DataFrame(albumIDtoRef))
    cols += ["Ref"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Artists")
    discdf = discdf.join(DataFrame(albumIDToArtists))
    cols += ["Artists"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)
    
    return discdf





########################################################################################################################
#
# Master DB Join
#
########################################################################################################################
def createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB):
    start, cmt = clock("\n=================================== Creating Artist ID DB ===================================")
    print("Creating Pandas DataFrame for {0} Arist IDs".format(discArtistDB.shape[0]))
    print("  Joining Artist Metadata")
    discdf = discArtistDB.join(discArtistMetadataDB)
    print("\tShape --> {0}".format(discdf.shape))
    print("  Joining Artist Albums")
    discdf = discdf.join(discArtistAlbumKnownDB)
    print("\tShape --> {0}".format(discdf.shape))
    elapsed(start, cmt)

    savename = disc.getMasterDiscogsDBFilename()
    saveFile(idata=discdf, ifile=savename, debug=True)
    
    
    
    
    
    
########################################################################################################################
#
# Associated Functions
#
########################################################################################################################
def directoryName(x):
    if x is None:
        return x
    if "..." in x:
        x = x.replace("...", "")
    if "/" in x:
        x = x.replace("/", "-")
    return x

def realName(x):
    if x is None:
        return [None,-1]
    
    lenx = len(x)
    if len(x) < 1:
        return [x,-1]

    if x[-1] != ")":
        return [x, None]
    

    if lenx >=5:
        if x[-3] == "(":
            try:
                num = int(x[-2:-1])
                val = x[:-3].strip()
                return [val, num]
            except:
                return [x, None]
            
    if lenx >= 6:
        if x[-4] == "(":
            try:
                num = int(x[-3:-1])
                val = x[:-4].strip()
                return [val, num]
            except:
                return [x, None]
            
    if lenx >= 7:
        if x[-4] == "(":
            try:
                num = int(x[-3:-1])
                val = x[:-4].strip()
                return [val, num]
            except:
                return [x, None]

    return [x, None]

def discConv(x):
    if x is None:
        return ""
    x = x.replace("/", "-")
    x = x.replace("¡", "")
    while x.startswith(".") and len(x) > 1:
        x = x[1:]
    x = x.strip()
    return x

def cleanMB(x):
    pos = [x.rfind("(")+1, x.rfind(")")]
    if sum([p > 0 for p in pos]) != len(pos):
        return x
    parval = x[pos[0]:pos[1]]
    return x[:pos[0]-2].strip()

# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

In [ ]:
%load_ext autoreload
%autoreload

from lookup import createArtistIDMap
from masterdb import getSlimArtistDB
from lookup import createArtistAlbumIDMap
from masterdb import getArtistAlbumsDB

dbs = dbdata.keys()
dbs = ["AllMusic"]

########################################################################################################
## Loop over DBs
########################################################################################################
for db in dbs:
    print("\n")
    print("="*125)
    print("="*125)
    print("=",db)
    print("="*125)
    print("="*125)
    print("\n")
    
    
    ####################################################################################################
    ## Create Artist ID Lookup
    ####################################################################################################
    createArtistIDMap(dbdata[db]["Disc"])
    
    
    ####################################################################################################
    ## Create Artist ID Pandas DataFrame
    ####################################################################################################
    discSlimArtistAMDB      = getSlimArtistDB(dbdata[db]["Disc"], force=True)
    discKnownSlimArtistAMDB = getKnownSlimArtistDB(dbdata[db]["Disc"], mydb, force=True)


    ####################################################################################################
    ## Create Artist ID to Album ID Lookup
    ####################################################################################################
    createArtistAlbumIDMap(dbdata[db]["Disc"])


    ####################################################################################################
    ## Create Artist ID Pandas DataFrame
    ####################################################################################################
    discArtistAMAlbumsDB = getArtistAlbumsDB(dbdata[db]["Disc"], force=True)

In [ ]:
%load_ext autoreload
%autoreload
from myMusicDBMap import myMusicDBMap
from artistDB import artistDB

mdb = myMusicDBMap(debug=True)
mydb = mdb.get()

***
***
***

## Artist ID --> Genre, Style, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discDC)

## Album ID --> Name, Ref, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

In [ ]:

discArtistLMAlbumsDB = getArtistAlbumsDB(discRM, force=False)

In [ ]:
from collections import Counter
cntr = Counter()
for artistData in discArtistLMAlbumsDB["Albums"]:
    for key in artistData.keys():
        cntr[key] += 1
cntr

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [ ]:
from masterdb import getArtistDB

In [ ]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

## Artist Metadata DB

In [ ]:
from masterdb import getArtistMetadataDB

In [ ]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

## Artist Albums DB

In [ ]:
from masterdb import getArtistAlbumsDB

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

## Albums DB

In [ ]:
from masterdb import getAlbumDB

In [ ]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

## Artist Album ID --> Known Albums

In [ ]:
from masterdb import getArtistAlbumKnownDB

In [ ]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random

# Download Artist Data

In [ ]:
artistName = "Killin' Baudelaire"

In [ ]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [ ]:
for i,artistName in enumerate(mdb.getArtists()):
        if artistName == "Riff Raff":
            print(i)
            break

In [ ]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 2383)

In [ ]:
# 3404